# Tutorial 7 - Parallelisation

While python gives us the flexibility to easily run complex paramateric sweeps, we now want to reach for better eficiency by introducing multiprocessing.

*This notebook is designed and run using linux.* \
*Getting multiprocessing working in a juypiter notebook on other platforms (e.g., windows) might be tricky.*

### Set Up

Lets get some intital imports out the way, and set auto reload, incase we change any of the external utils etc.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

Now, `pyspice` imports, and setting up a logger.

In [ ]:
import PySpice
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit
from PySpice.Unit import *

logger = Logging.setup_logging()

Might need to change the simulator...

In [ ]:
# # # change sim program location depending on system
# if sys.platform == "linux" or sys.platform == "linux2":
#     PySpice.Spice.Simulation.CircuitSimulator.DEFAULT_SIMULATOR = 'ngspice-subprocess'  # needed for linux
# elif sys.platform == "win32":
#     # You will get logging errors/warning, but is should work
#     pass

Lets define several circuits for our analysis to iterate over.

In [ ]:
my_sweep = []
for r in np.arange(500, 100000, 500):

    # # create the circuit
    circuit = Circuit(f"Tutorial 7: R={r} Ohm")

    # # add components to the circuit
    # circuit.V('input', 'n1', circuit.gnd, 10@u_V) # DC voltage comonent
    Vac = circuit.SinusoidalVoltageSource('input', 'n1', circuit.gnd, amplitude=1@u_V, frequency=100@u_Hz)
    R = circuit.R(1, 'n1', 'n2', r@u_kOhm)  # @u_kΩ is a unit of kOhms
    C = circuit.C(1, 'n2', circuit.gnd, 1@u_uF)
    circuit.Diode(1, 'n2', 'n3', model='MyDiode')  # using cutom defined diode
    circuit.R(2, 'n3', circuit.gnd, 1@u_kOhm)  # @u_kΩ is a unit of kOhms

    # add our diode
    circuit.model('MyDiode', 'D', IS=4.352@u_nA, RS=0.6458@u_Ohm, BV=110@u_V, IBV=0.0001@u_V, N=1.906)  # Define the 1N4148PH (Signal Diode)

    # Print the netlist
    simulator = circuit.simulator(temperature=25, nominal_temperature=25)

    my_sweep.append(simulator)

Set up simulator function

In [ ]:
def perform_simulation(simulation):
    analysis = simulator.transient(step_time=0.0001, end_time=0.1)
    return analysis

In [ ]:
import time

tic = time.time()
results = [perform_simulation(sim) for sim in my_sweep]

toc = time.time()
print(f"Total time = {toc-tic}")

Now, if we just thow this directly into a multiprocessing pool, it fails...

We can see it has something to do with `ForkingPickler`, so we can assume this is when we pass the simulator into, or the analysis result out of, the process. 

In [ ]:
from multiprocessing import Pool

if __name__ ==  '__main__': 
    
    try:
        with Pool() as p:
            results_mp = p.map(perform_simulation, my_sweep)
    except Exception as e:
        print('Failed to MP with passed sims')
  

Lets format the analysis results ans see if that helps

In [ ]:
from utils.methods import format_analysis

def perform_simulation_to_dict(simulator) -> dict:
    analysis = simulator.transient(step_time=0.0001, end_time=0.1)
    return format_analysis(analysis)

This still fails...

In [ ]:
from multiprocessing import Pool

if __name__ ==  '__main__': 
    
    try:
        with Pool() as p:
            results_mp = p.map(perform_simulation_to_dict, my_sweep)
    except Exception as e:
        print('Failed to MP with passed sims with a formatted output')
  

Finally, lets look at creating the circuit, running the simulation, and extracting the desired results which can be returned without an error.

In [ ]:
def create_and_perform_simulation(r:float) -> dict:

    # # create the circuit
    circuit = Circuit(f"Tutorial 7: R={r} Ohm")

    # # add components to the circuit
    # circuit.V('input', 'n1', circuit.gnd, 10@u_V) # DC voltage comonent
    Vac = circuit.SinusoidalVoltageSource('input', 'n1', circuit.gnd, amplitude=1@u_V, frequency=100@u_Hz)
    R = circuit.R(1, 'n1', 'n2', r@u_kOhm)  # @u_kΩ is a unit of kOhms
    C = circuit.C(1, 'n2', circuit.gnd, 1@u_uF)
    circuit.Diode(1, 'n2', 'n3', model='MyDiode')  # using cutom defined diode
    circuit.R(2, 'n3', circuit.gnd, 1@u_kOhm)  # @u_kΩ is a unit of kOhms

    # add our diode
    circuit.model('MyDiode', 'D', IS=4.352@u_nA, RS=0.6458@u_Ohm, BV=110@u_V, IBV=0.0001@u_V, N=1.906)  # Define the 1N4148PH (Signal Diode)

    # Print the netlist
    simulator = circuit.simulator(temperature=25, nominal_temperature=25)

    analysis = simulator.transient(step_time=0.0001, end_time=0.1)

    return format_analysis(analysis)

In [ ]:
resistor_sweep = np.arange(500, 100000, 500)

if __name__ ==  '__main__': 
    
    tic = time.time()
    with Pool() as p:
        results_mp = p.map(create_and_perform_simulation, resistor_sweep)
    toc = time.time()
    
mp_time = toc-tic
print(f"MP Total time = {mp_time}")

For a direct comparison lets run serially.

We get much faster speeds with multiprocessing.

In [ ]:
tic = time.time()
results_serial = [create_and_perform_simulation(r) for r in resistor_sweep]
toc = time.time()

serial_time = toc-tic
print(f"Serially run total time = {serial_time}")

speed_up = serial_time/mp_time
f"Speed up is: {speed_up:.2f} times"

Now, the results from the multiprocessing and just manually looping though should be the same.
Lets test this:

In [ ]:
for i, res_serial in enumerate(results_serial):

    for k, v in res_serial.items():
        
        are_they_equal = (results_mp[i][k] == v).all()

        if are_they_equal is False:
            raise ValueError('Serial and Multiprocessed results are not the same')

'Passed test!'

### Summary

In order to exploit multiprocessing, the circuit, simulation, and results extracted from the analysis must all be performed within the spun up process.
i.e., PySpice objects cannot currently be passed to and from processes.